In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
!pip install -U pip wheel setuptools
!pip install concrete-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━

In [5]:
import numpy as np
from concrete import fhe

# Generate synthetic data
np.random.seed(42)
X = np.random.rand(5, 2)  # 5 samples, 2 features
y = X @ np.array([3, 5]) + np.random.randn(5) * 0.1  # Linear relation with noise

# Add bias column
X_bias = np.c_[np.ones(X.shape[0]), X]

# Convert to integer format
scaling_factor = 5  # Reduce scaling to stay within 16-bit FHE limits
X_int = np.clip((X_bias * scaling_factor).astype(np.uint8), 0, 255)
y_int = np.clip((y * scaling_factor).astype(np.uint8), 0, 255)

# Define encrypted Least Squares using integer gradient descent
def fhe_gradient_descent(X_enc, y_enc, lr=1, epochs=5):
    m, n = X_enc.shape
    theta = np.zeros(n, dtype=np.uint8)  # Use uint8 to prevent bit growth

    for _ in range(epochs):
        pred = np.matmul(X_enc, theta)
        error = np.add(pred, -y_enc)  # Keep addition simple
        gradient = np.matmul(X_enc.T, error) >> 3  # Bitwise shift instead of division
        theta = np.add(theta, -lr * gradient)  # Prevent overflow

    return theta  # NumPy array

# Define Concrete function
@fhe.compiler({"X_enc": "encrypted", "y_enc": "encrypted"})
def fhe_least_squares(X_enc, y_enc):
    return fhe_gradient_descent(X_enc, y_enc)

# Compile FHE circuit
inputset = [(X_int, y_int)]
circuit = fhe_least_squares.compile(inputset)

# Encrypt, run, and decrypt FHE computation
encrypted_result = circuit.encrypt_run_decrypt(X_int, y_int)

# Scale back results
theta_fhe = encrypted_result / scaling_factor
print("With FHE (Concrete) coefficients:", theta_fhe)


With FHE (Concrete) coefficients: [-324.4  -18.8 -262.2]
